In [ ]:
# === SHAP on a Random Forest (fast, robust) — explain test instance #15 ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, classification_report,
    confusion_matrix, RocCurveDisplay
)

from scipy import sparse
import shap
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# ---------------------------
# 1) Load data
# ---------------------------
excel_file = pd.ExcelFile("Scoring_Database.xlsx")
df = excel_file.parse("Data")

target = "Default"
X = df.drop(columns=[target])
y = df[target]

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# ---------------------------
# 2) Preprocess + RandomForest
# ---------------------------
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

numeric_transformer = SimpleImputer(strategy="median")
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ],
    remainder="drop"
)

# Smaller forest for speed
rf = RandomForestClassifier(
    n_estimators=200,            # reduced for faster training
    max_depth=None,
    n_jobs=-1,
    class_weight="balanced_subsample",
    random_state=42
)

clf = Pipeline(steps=[("prep", preprocess), ("rf", rf)])
clf.fit(X_train, y_train)

# ---------------------------
# 3) Performance
# ---------------------------
proba_test = clf.predict_proba(X_test)[:, 1]
y_pred = (proba_test >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, proba_test)

print(f"Accuracy (test): {acc:.4f}")
print(f"ROC-AUC  (test): {auc:.4f}\n")

print("Confusion matrix (test):")
print(confusion_matrix(y_test, y_pred), "\n")

print("Classification report (test):")
print(classification_report(y_test, y_pred, digits=3))

RocCurveDisplay.from_predictions(y_test, proba_test)
plt.title("Random Forest — ROC (test)")
plt.tight_layout()
plt.show()

# ---------------------------
# 4) SHAP explanations (fast, robust)
# ---------------------------
prep = clf.named_steps["prep"]
X_train_tr = prep.transform(X_train)
X_test_tr  = prep.transform(X_test)

# Dense arrays for SHAP
if sparse.issparse(X_train_tr): X_train_tr = X_train_tr.toarray()
if sparse.issparse(X_test_tr):  X_test_tr  = X_test_tr.toarray()

# Transformed feature names (robust)
def get_transformed_feature_names(prep_ct, num_cols, cat_cols):
    try:
        names = list(prep_ct.get_feature_names_out())
        return [n.split("__", 1)[-1] for n in names]
    except Exception:
        names = list(num_cols)
        if len(cat_cols) > 0 and "cat" in prep_ct.named_transformers_:
            cat_pipe = prep_ct.named_transformers_["cat"]
            if hasattr(cat_pipe, "named_steps") and "onehot" in cat_pipe.named_steps:
                ohe = cat_pipe.named_steps["onehot"]
                if hasattr(ohe, "categories_"):
                    if hasattr(ohe, "get_feature_names_out"):
                        names += list(ohe.get_feature_names_out(cat_cols))
                    else:
                        for col, cats in zip(cat_cols, ohe.categories_):
                            names += [f"{col}_{str(c)}" for c in cats]
        return names

feature_names_tr = get_transformed_feature_names(prep, num_cols, cat_cols)

# Compact background for SHAP (faster)
background = shap.utils.sample(X_train_tr, 100, random_state=42)

# Use the UNIFIED Explainer to avoid version-specific returns
explainer = shap.Explainer(clf.named_steps["rf"], background)

# Positive class index for "Default" (in case values are multi-output)
classes = list(clf.named_steps["rf"].classes_)
pos_idx = classes.index(1) if 1 in classes else (classes.index(True) if True in classes else (1 if len(classes) > 1 else 0))

# ----- Local SHAP on test instance #15 -----
idx_inst = min(15, X_test_tr.shape[0]-1)      # safe clamp
print(f"\nExplaining test instance at positional index: {idx_inst}")
x_instance = X_test_tr[idx_inst:idx_inst+1]   # (1, n_features)

ex_inst = explainer(x_instance)               # Explanation
vals = ex_inst.values[0]                      # could be (n_features,) or (n_features, n_outputs)
base = ex_inst.base_values[0]
xdat = ex_inst.data[0]

# If multi-output, select the positive class on the last dimension
if vals.ndim == 2:
    vals = vals[:, pos_idx]
    if np.ndim(base) > 0:
        base = base[pos_idx]

# (A) Force plot — new API: base value first
shap.plots.force(
    float(base),                # scalar base value
    vals.astype(float),         # 1D shap values
    features=xdat.astype(float),
    feature_names=feature_names_tr,
    matplotlib=True,
    show=True
)

# (B) Waterfall plot
ex_one = shap.Explanation(values=vals, base_values=base, data=xdat,
                          feature_names=feature_names_tr)
shap.plots.waterfall(ex_one, max_display=12, show=True)

# ----- Global SHAP on a random subset of the test set (fast) -----
rng = np.random.default_rng(7)
sub_n = min(300, X_test_tr.shape[0])          # cap for speed
idx_sub = rng.choice(X_test_tr.shape[0], size=sub_n, replace=False)
X_test_sub = X_test_tr[idx_sub]

ex_sub = explainer(X_test_sub)                # Explanation
vals_sub = ex_sub.values                      # (n_samples, n_features) or (n_samples, n_features, n_outputs)
if vals_sub.ndim == 3:
    vals_sub = vals_sub[:, :, pos_idx]

# (C) Summary (beeswarm)
plt.figure()
shap.summary_plot(vals_sub, X_test_sub, feature_names=feature_names_tr, show=True)

# (D) SHAP bar "feature importance" (mean |SHAP|)
plt.figure()
shap.summary_plot(vals_sub, X_test_sub, feature_names=feature_names_tr, plot_type="bar", show=True)
